In [14]:
import pandas as pd
import glob
import re
from const import COlS
from datetime import date, datetime
import logging


In [15]:
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
# Create a logger
logger = logging.getLogger(__name__)

In [16]:
logger.info('Importing file Map Owner...')
Rep_files = glob.glob("./input/*.csv")
Map_path = glob.glob("./input/Map*.xlsx")
Map_owner = pd.read_excel(Map_path[0], engine='openpyxl').astype(str)

2025-03-06 11:15:59,282 - INFO - Importing file Map Owner...


In [17]:
logger.info('Importing file Map Owner...')
Rep_files = glob.glob("./input/*.csv")
Map_path = glob.glob("./input/Map*.xlsx")
Map_owner = pd.read_excel(Map_path[0], engine='openpyxl').astype(str)
Map_owner = Map_owner.drop(columns={'Assign Date'})
Map_owner = Map_owner.rename(columns={'Loan No.': 'Loan No', 'Owner': 'OA', 'Due': 'DUE'})
logger.info('Map Owner impoerted done!!')

file_date = re.findall(r'\d+', Rep_files[0])[0]
file_time = re.findall(r'\d+', Rep_files[0])[1]
logger.info('Importing Rep file...')

Rep = pd.read_csv(Rep_files[0], usecols=COlS.use_cols, encoding='cp874').astype(str)
Rep = Rep.merge(Map_owner, on='Loan No', how='left')
logger.info('Rep file impoerted done!!')
logger.info('Start process cleanning and formatting files...')

2025-03-06 11:16:04,604 - INFO - Importing file Map Owner...
2025-03-06 11:16:10,079 - INFO - Map Owner impoerted done!!
2025-03-06 11:16:10,079 - INFO - Importing Rep file...
2025-03-06 11:16:10,536 - INFO - Rep file impoerted done!!
2025-03-06 11:16:10,549 - INFO - Start process cleanning and formatting files...


In [18]:
payment_path = glob.glob("./output/*.xlsx")
print(payment_path)


['./output\\02.All Payment Feb 2025.xlsx']


In [19]:
def read_all_sheets(file_path):
    xls = pd.ExcelFile(file_path, engine="openpyxl")  # Load the Excel file
    all_sheets = []  # List to store individual DataFrames

    for sheet_name in xls.sheet_names:  # Iterate over all sheets
        print(f"Reading sheet: {sheet_name}")  # Debugging statement
        df = pd.read_excel(xls, sheet_name=sheet_name, dtype=str)  # Read each sheet as string
        all_sheets.append(df)

    # Combine all DataFrames into one
    if all_sheets:
        final_df = pd.concat(all_sheets, ignore_index=True)
    else:
        final_df = pd.DataFrame()  # Empty DataFrame if no sheets are found
    
    return final_df

In [23]:
payment_path = glob.glob("./output/*.xlsx")[0]


In [24]:
payment = read_all_sheets(payment_path)

Reading sheet: Sheet1


In [27]:
payment

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN
1,1000163100000476,20250201,20250201,1,11333022:BBL(Realtime),08:Payment for written-off Loan,3001,RC120250201027336551,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
2,1000163110000240,20250201,20250131,1,11333022:BBL(Realtime),01:Normal Payment,4224,RC120250131027270408,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
3,1000163110000444,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,6376,RC120250201027322694,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
4,1000163120000064,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,4694,RC120250201027620545,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
959944,9010221092001489,20250227,20250227,1,11333022:BBL(Realtime),04:Partial Early Payment,2055,RC120250227028517105,A501:TL,CFNCRC0109B,NaN,NaN,NaN
959945,9010221092001721,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,1100,RC120250227028520017,A501:TL,CFNCRC0109B,GMV,1,NaN
959946,9010221102000854,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,2037,RC120250227028544833,A501:TL,CFNCRC0109B,DHA,1,NaN
959947,9010221102001648,20250227,20250227,1,11333022:BBL(Realtime),04:Partial Early Payment,1887,RC120250227028562426,A501:TL,CFNCRC0109B,DHA,1,NaN


In [ ]:
common_duplicate = payment[payment['Receipt Number'].isin(Rep['Receipt Number'])]
common_duplicate.groupby('Receipt date').size()
common_duplicate.to_excel('duplicated_27&28.xlsx', index=False, engine='xlsxwriter')
Rep.groupby('Receipt date').size()
common_unique = payment[~payment['Receipt Number'].isin(Rep['Receipt Number'])]
common_unique.groupby('Receipt date').size()
common_unique = Rep[~Rep['Receipt Number'].isin(payment['Receipt Number'])]
common_unique.groupby('Receipt date').size()
common_unique.to_excel('unique_28.xlsx', index=False, engine='xlsxwriter')


In [28]:
payment_combine = pd.concat([payment, Rep])
payment_combine

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN
1,1000163100000476,20250201,20250201,1,11333022:BBL(Realtime),08:Payment for written-off Loan,3001,RC120250201027336551,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
2,1000163110000240,20250201,20250131,1,11333022:BBL(Realtime),01:Normal Payment,4224,RC120250131027270408,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
3,1000163110000444,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,6376,RC120250201027322694,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
4,1000163120000064,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,4694,RC120250201027620545,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83605,9010221042000039,20250226,20250226,1,11333022:BBL(Realtime),01:Normal Payment,2221.32,RC120250226028447688,A501:TL,CFNCRC0109B,KWC,1,NaN
83606,9010221042000080,20250226,20250226,1,11333022:BBL(Realtime),01:Normal Payment,5.0,RC120250226028469243,A501:TL,CFNCRC0109B,NaN,NaN,NaN
83607,9010221042000080,20250226,20250226,2,:,02:Early Payment,0.0,RC120250226028471194,A501:TL,6507015,NaN,NaN,NaN
83608,9010221112000199,20250226,20250226,1,11333022:BBL(Realtime),01:Normal Payment,5000.0,RC120250226028476627,A501:TL,CFNCRC0109B,UPC,1,NaN


In [41]:
payment.groupby('Receipt date').size()


Receipt date
20250201    306606
20250202    158452
20250203     84906
20250204    110786
20250205     30077
20250206     13596
20250207     10944
20250208      6668
20250209      4669
20250210      9262
20250211      5155
20250212      4048
20250213      4381
20250214      5052
20250215      5274
20250216      4932
20250217      6234
20250218      5286
20250219      5051
20250220      9481
20250221      8124
20250222      6839
20250223      6662
20250224      9572
20250225     45826
20250227     92066
dtype: int64

In [29]:
Rep.groupby('Receipt date').size()


Receipt date
20250226    83610
dtype: int64

In [30]:
payment_combine

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN
1,1000163100000476,20250201,20250201,1,11333022:BBL(Realtime),08:Payment for written-off Loan,3001,RC120250201027336551,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
2,1000163110000240,20250201,20250131,1,11333022:BBL(Realtime),01:Normal Payment,4224,RC120250131027270408,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
3,1000163110000444,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,6376,RC120250201027322694,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
4,1000163120000064,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,4694,RC120250201027620545,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83605,9010221042000039,20250226,20250226,1,11333022:BBL(Realtime),01:Normal Payment,2221.32,RC120250226028447688,A501:TL,CFNCRC0109B,KWC,1,NaN
83606,9010221042000080,20250226,20250226,1,11333022:BBL(Realtime),01:Normal Payment,5.0,RC120250226028469243,A501:TL,CFNCRC0109B,NaN,NaN,NaN
83607,9010221042000080,20250226,20250226,2,:,02:Early Payment,0.0,RC120250226028471194,A501:TL,6507015,NaN,NaN,NaN
83608,9010221112000199,20250226,20250226,1,11333022:BBL(Realtime),01:Normal Payment,5000.0,RC120250226028476627,A501:TL,CFNCRC0109B,UPC,1,NaN


In [31]:
payment_combine= payment_combine.sort_values(by='Receipt date')
payment_combine.groupby('Receipt date').size()

Receipt date
20250201    306606
20250202    158452
20250203     84906
20250204    110786
20250205     30077
20250206     13596
20250207     10944
20250208      6668
20250209      4669
20250210      9262
20250211      5155
20250212      4048
20250213      4381
20250214      5052
20250215      5274
20250216      4932
20250217      6234
20250218      5286
20250219      5051
20250220      9481
20250221      8124
20250222      6839
20250223      6662
20250224      9572
20250225     45826
20250226     83610
20250227     92066
dtype: int64

In [32]:
payment_combine['Payment Amt(Total payment Amt)'] = payment_combine['Payment Amt(Total payment Amt)'].astype('float64')
payment_combine

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017.00,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN
204407,7240870103634537,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,973.12,RC120250201027600384,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
204406,7240870103634524,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,414.07,RC120250201027771530,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN
204405,7240870103634512,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,1028.03,RC120250201027422685,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
204404,7240870103634507,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,947.40,RC120250201027695318,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
929262,7240970103760860,20250227,20250227,1,11333026:M-pay(Realtime SFP),01:Normal Payment,828.21,RC120250227028548973,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
929263,7240970103760889,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,754.49,RC120250227028526259,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN
929264,7240970103761004,20250227,20250227,1,11333026:M-pay(Realtime SFP),01:Normal Payment,719.61,RC120250227028543081,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
929244,7240970103760307,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,367.32,RC120250227028588814,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN


In [33]:
# Drop rows where 'number' is duplicated
payment_unique = payment_combine[payment_combine.duplicated('Receipt Number', keep=False) == False]
payment_unique

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017.00,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN
204407,7240870103634537,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,973.12,RC120250201027600384,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
204406,7240870103634524,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,414.07,RC120250201027771530,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN
204405,7240870103634512,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,1028.03,RC120250201027422685,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
204404,7240870103634507,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,947.40,RC120250201027695318,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
929262,7240970103760860,20250227,20250227,1,11333026:M-pay(Realtime SFP),01:Normal Payment,828.21,RC120250227028548973,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
929263,7240970103760889,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,754.49,RC120250227028526259,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN
929264,7240970103761004,20250227,20250227,1,11333026:M-pay(Realtime SFP),01:Normal Payment,719.61,RC120250227028543081,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
929244,7240970103760307,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,367.32,RC120250227028588814,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN


In [34]:
payment_res = payment_unique[payment_unique['Payment Amt(Total payment Amt)'].astype('float64') > 0]
payment_res.groupby('Receipt date').size()

Receipt date
20250201    306606
20250202    158452
20250203     84906
20250204    110786
20250205     30077
20250206     13596
20250207     10944
20250208      6668
20250209      4669
20250210      9262
20250211      5155
20250212      4048
20250213      4381
20250214      5052
20250215      5274
20250216      4932
20250217      6234
20250218      5286
20250219      5051
20250220      9481
20250221      8124
20250222      6839
20250223      6662
20250224      9572
20250225     45826
20250226     83529
20250227     92066
dtype: int64

In [35]:
payment_res

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017.00,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN
204407,7240870103634537,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,973.12,RC120250201027600384,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
204406,7240870103634524,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,414.07,RC120250201027771530,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN
204405,7240870103634512,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,1028.03,RC120250201027422685,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
204404,7240870103634507,20250201,20250201,1,11333026:M-pay(Realtime SFP),01:Normal Payment,947.40,RC120250201027695318,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
929262,7240970103760860,20250227,20250227,1,11333026:M-pay(Realtime SFP),01:Normal Payment,828.21,RC120250227028548973,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
929263,7240970103760889,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,754.49,RC120250227028526259,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN
929264,7240970103761004,20250227,20250227,1,11333026:M-pay(Realtime SFP),01:Normal Payment,719.61,RC120250227028543081,A701:SFLoan,CFNCIF1505,NaN,NaN,NaN
929244,7240970103760307,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,367.32,RC120250227028588814,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN


In [36]:
payment_final = payment_res[~payment_res['Receipt channel'].str.contains('CIMB')].replace('nan', None).sort_values(by='Receipt date')
payment_final

,Loan No,Receipt date,Payment date,Receipt Seq No,Receipt channel,Payment type,Payment Amt(Total payment Amt),Receipt Number,Sales product code,Last processed user No,OA,DUE,Remark
0,1000163020001077,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2017.00,RC120250201027669716,A501:TL,CFNCRC0109B,NaN,NaN,NaN
15,5000159071005450,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,2000.00,RC120250201027275293,A301:RL,CFNCRC0109B,NaN,NaN,NaN
153302,7240670103101814,20250201,20250201,1,11333022:BBL(Realtime),01:Normal Payment,644.14,RC120250201027541218,A701:SFLoan,CFNCRC0109B,NaN,NaN,NaN
1,1000163100000476,20250201,20250201,1,11333022:BBL(Realtime),08:Payment for written-off Loan,3001.00,RC120250201027336551,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
2,1000163110000240,20250201,20250131,1,11333022:BBL(Realtime),01:Normal Payment,4224.00,RC120250131027270408,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
867887,1000163100000689,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,12752.00,RC120250227028518265,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
867886,1000163080000352,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,7000.00,RC120250227028514465,A101:C4C,CFNCRC0109B,DHA,1,NaN
867885,1000163040000122,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,19351.00,RC120250227028542890,A101:C4C,CFNCRC0109B,NaN,NaN,NaN
867927,5000159081067491,20250227,20250227,1,11333022:BBL(Realtime),01:Normal Payment,1300.00,RC120250227028527633,A301:RL,CFNCRC0109B,GMV,1,NaN


In [37]:
payment_final.groupby('Receipt date').size()

Receipt date
20250201    306606
20250202    158452
20250203     84906
20250204    110786
20250205     30077
20250206     13596
20250207     10944
20250208      6668
20250209      4669
20250210      9262
20250211      5155
20250212      4048
20250213      4381
20250214      5052
20250215      5274
20250216      4932
20250217      6234
20250218      5286
20250219      5051
20250220      9481
20250221      8124
20250222      6839
20250223      6662
20250224      9572
20250225     45826
20250226     83529
20250227     92066
dtype: int64

In [38]:
import pandas as pd

def auto_split_payment_data(payment_final, range_size=5):
    # Convert the 'date' column to datetime if it's not already in that format
    payment_final['datechunk'] = pd.to_datetime(payment_final['Receipt date'])
    
    # Get the day of the month from the 'date' column
    payment_final['datechunk'] = payment_final['datechunk'].dt.day
    
    # Get the maximum day value in the data to determine the number of ranges
    max_day = payment_final['datechunk'].max()

    # Create a dictionary to store the splits
    payment_splits = {}

    # Loop through the days in the specified range and create splits
    for start_day in range(1, max_day + 1, range_size):
        end_day = min(start_day + range_size - 1, max_day)
        pay_key = f'pay{(start_day - 1) // range_size + 1}'
        
        # Filter payment data based on the current day range
        payment_splits[pay_key] = payment_final[(payment_final['datechunk'] >= start_day) & 
                                                (payment_final['datechunk'] <= end_day)]

    return payment_splits




In [39]:
def save_payment_splits_to_excel(payment_final, file_name, range_size=5):
    # Split the data dynamically
    payment_splits = auto_split_payment_data(payment_final, range_size)
    # output_dir='./output/'
    # os.makedirs(output_dir, exist_ok=True)
    
    # Save the splits to an Excel file
    with pd.ExcelWriter(f'./output/{file_name}', engine='xlsxwriter') as writer:
        # Loop through the payment splits and save each as a sheet
        for pay_key, pay_data in payment_splits.items():
            print('pay_data',len(pay_data))
            # Save the split data to an Excel sheet with the sheet name based on the pay_key
            pay_data.to_excel(writer, index=False, sheet_name=pay_key, engine='xlsxwriter')
    # for pay_key, pay_data in payment_splits.items():
    #     file_name = f"{output_dir}{pay_key}.xlsx"  # Filename based on pay_key
    #     print(f"Saving {len(pay_data)} rows to file: {file_name}")
        
    #     # Save the split data to an Excel file
    #     pay_data.to_excel(file_name, index=False, engine='xlsxwriter')



In [40]:
file_name = 'payment_splits.xlsx' 
save_payment_splits_to_excel(payment_final, file_name,5)

pay_data 690827
pay_data 45139
pay_data 23910
pay_data 30984
pay_data 77023
pay_data 175595


In [239]:
file_name = payment_path[0].split('\\')[1]
file_name

'payment_splits.xlsx'

In [ ]:
payment.sum()

In [ ]:
len(payment)